In [39]:
# adapted from https://docs.aws.amazon.com/lambda/latest/dg/with-s3-tutorial.html
import os
import typing
import uuid
from urllib.parse import unquote_plus
os.environ["AWS_DEFAULT_REGION"] =  "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "test"
os.environ["AWS_SECRET_ACCESS_KEY"] = "test"

import boto3
from PIL import Image

if typing.TYPE_CHECKING:
    from mypy_boto3_s3 import S3Client
    from mypy_boto3_ssm import SSMClient
    from mypy_boto3_dynamodb import DynamoDBClient, DynamoDBServiceResource

MAX_DIMENSIONS = 400, 400
"""The max width and height to scale the image to."""

endpoint_url = None
if os.getenv("STAGE") == "local":
    endpoint_url = "https://localhost.localstack.cloud:4566"

args = {
    "region_name": "us-east-1",
    "endpoint_url": "http://localhost:4566",
    "aws_access_key_id": "test",
    "aws_secret_access_key": "test",
}
s3: "S3Client" = boto3.client("s3", **args)
ssm: "SSMClient" = boto3.client("ssm", **args)
client: "DynamoDBServiceResource" = boto3.client("dynamodb", **args)
dynamodb: "DynamoDBClient" = boto3.resource("dynamodb", **args)

In [49]:
ssm.describe_parameters()
ssm.describe_parameters()

{'Parameters': [{'Name': '/localstack-thumbnail-app/buckets/images',
   'Type': 'String',
   'LastModifiedDate': datetime.datetime(2025, 6, 6, 15, 55, 56, 734000, tzinfo=tzlocal()),
   'LastModifiedUser': 'N/A',
   'Version': 1,
   'DataType': 'text'},
  {'Name': '/localstack-thumbnail-app/buckets/resized',
   'Type': 'String',
   'LastModifiedDate': datetime.datetime(2025, 6, 6, 15, 56, 2, 625000, tzinfo=tzlocal()),
   'LastModifiedUser': 'N/A',
   'Version': 1,
   'DataType': 'text'},
  {'Name': '/localstack--assignment3/buckets/reviewsraw',
   'Type': 'String',
   'LastModifiedDate': datetime.datetime(2025, 6, 6, 17, 13, 25, 744000, tzinfo=tzlocal()),
   'LastModifiedUser': 'N/A',
   'Version': 1,
   'DataType': 'text'},
  {'Name': '/localstack-assignment3/buckets/reviewsprocessed',
   'Type': 'String',
   'LastModifiedDate': datetime.datetime(2025, 6, 6, 17, 13, 27, 159000, tzinfo=tzlocal()),
   'LastModifiedUser': 'N/A',
   'Version': 1,
   'DataType': 'text'},
  {'Name': '/locals

In [51]:
for parameter in ssm.describe_parameters()['Parameters']:
    print(parameter['Name'])

/localstack-thumbnail-app/buckets/images
/localstack-thumbnail-app/buckets/resized
/localstack--assignment3/buckets/reviewsraw
/localstack-assignment3/buckets/reviewsprocessed
/localstack-assignment3/tables/sentiment
/localstack-assignment3/tables/profanity
/localstack-assignment3/tables/users


In [40]:
def get_table_name() -> str:
    parameter = ssm.get_parameter(Name="/localstack-assignment3/tables/profanity")
    return parameter["Parameter"]["Value"]

In [41]:
get_table_name()

'Profanity'

In [42]:
client.list_tables()

{'TableNames': ['Profanity', 'Sentiment', 'Users'],
 'ResponseMetadata': {'RequestId': '77d7c00e-a532-4277-aa34-17d12adefa30',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'TwistedWeb/24.3.0',
   'date': 'Fri, 06 Jun 2025 15:32:36 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '51',
   'x-amzn-requestid': '77d7c00e-a532-4277-aa34-17d12adefa30',
   'x-amz-crc32': '792070558'},
  'RetryAttempts': 0}}

In [45]:
table = dynamodb.Table("Profanity")

In [47]:
table.put_item(
    Item={
    "reviewerID": "review_0001.json",
    "has_profanity": True,
    "UserID": "Theresa Mayer",
    }
)

{'ResponseMetadata': {'RequestId': '6e2f34e0-7d91-4651-8f13-a65e4e572787',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'TwistedWeb/24.3.0',
   'date': 'Fri, 06 Jun 2025 15:34:22 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'x-amzn-requestid': '6e2f34e0-7d91-4651-8f13-a65e4e572787',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [54]:
table.get_item(Key={"reviewerID": "review_0001.json"})

{'Item': {'has_profanity': True,
  'UserID': 'Theresa Mayer',
  'ReviewID': 'review_0001.json'},
 'ResponseMetadata': {'RequestId': 'd71bb9d7-8ce7-40a8-ab07-9a856488b861',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'TwistedWeb/24.3.0',
   'date': 'Fri, 06 Jun 2025 15:39:20 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '116',
   'x-amzn-requestid': 'd71bb9d7-8ce7-40a8-ab07-9a856488b861',
   'x-amz-crc32': '3931046080'},
  'RetryAttempts': 0}}